In [2]:
%pip install tensorflow[and-cuda]
%pip install matplotlib
%pip install tables
%pip install pillow
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image 
import cv2
from random import shuffle
import glob
import tables
import cv2  
from math import ceil
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))


2024-12-23 19:45:29.081469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-23 19:45:29.092042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735001129.101652  167806 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735001129.105545  167806 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-23 19:45:29.117982: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available:  1


In [7]:
hdf5_path = "./data/AZdata.keras"
train_path = "./AZout/*/*.png"

In [8]:
addrs = glob.glob(train_path)
labels = [addr.split("/")[2] for addr in addrs]
unique = set(labels)
uniqueList = list(unique)
uniqueList.sort()
print(uniqueList)

for i in range(len(labels)):
    labels[i] = uniqueList.index(labels[i])

c = list(zip(addrs, labels))
shuffle(c)
addrs, labels = zip(*c)
train_addrs = addrs[0:int(0.6*len(addrs))]
train_labels = labels[0:int(0.6*len(labels))]

val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]

test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]
print("train size:",len(train_addrs))
print("val size:",len(val_addrs))
print("test size:",len(test_addrs))
print(labels[:5])

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
train size: 223470
val size: 74490
test size: 74491
(12, 21, 19, 20, 14)


In [ ]:
data_order = "tf"
img_dtype = tables.UInt8Atom() 


data_shape = (0, 32, 32, 3)

hdf5_file = tables.open_file(hdf5_path, mode="w")
try:
    train_storage = hdf5_file.create_earray(hdf5_file.root, "train_img", img_dtype, shape=data_shape)
    val_storage = hdf5_file.create_earray(hdf5_file.root, "val_img", img_dtype, shape=data_shape)
    test_storage = hdf5_file.create_earray(hdf5_file.root, "test_img", img_dtype, shape=data_shape)
    
    mean_storage = hdf5_file.create_earray(hdf5_file.root, "train_mean", img_dtype, shape=data_shape)
    
    hdf5_file.create_array(hdf5_file.root, "train_labels", train_labels)
    hdf5_file.create_array(hdf5_file.root, "val_labels", val_labels)
    hdf5_file.create_array(hdf5_file.root, "test_labels", test_labels)
    
    mean = np.zeros(data_shape[1:], np.float32)
    
    for i in range(len(train_addrs)):
        if i % 10 == 0 and i > 1:
            print("Train data: {}/{}".format(i, len(train_addrs)))
        addr = train_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        train_storage.append(img[None])
        mean += img / float(len(train_labels))
    
    for i in range(len(val_addrs)): 
        if i % 10 == 0 and i > 1:
            print ("Validation data: {}/{}".format(i, len(val_addrs)))
    

        addr = val_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        val_storage.append(img[None])
    
    for i in range(len(test_addrs)):
        if i % 10 == 0 and i > 1:
            print ("Test data: {}/{}".format(i, len(test_addrs)))

        addr = test_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        test_storage.append(img[None])
    
    mean_storage.append(mean[None])
    print("HDF5 Done")
finally: 
    print("In Finally")
    hdf5_file.close()

(32, 32)
In Finally


ValueError: the ranks of the appended object (2) and the ``/train_img`` EArray (4) differ

In [45]:
subtract_mean = True
batch_size = 50
nb_class = 314

In [46]:
hdf5_file = tables.open_file(hdf5_path, mode="r")
if subtract_mean:
    mm = hdf5_file.root.train_mean[0]
    mm = mm[np.newaxis, ...]

train_data = np.array(hdf5_file.root.train_img)
train_label = np.array(hdf5_file.root.train_labels)

test_data = np.array(hdf5_file.root.test_img)
test_label = np.array(hdf5_file.root.test_labels)

val_data = np.array(hdf5_file.root.val_img)
val_label = np.array(hdf5_file.root.val_labels)

print("train data:",train_data.shape," train_label",train_label.shape)
print("test_data:",test_data.shape," test_label:",test_label.shape)
print("val_data:",val_data.shape," val_label:",val_label.shape)

train data: (223470, 32, 32, 3)  train_label (223470,)
test_data: (74491, 32, 32, 3)  test_label: (74491,)
val_data: (74490, 32, 32, 3)  val_label: (74490,)


In [47]:
from tensorflow.keras.utils import to_categorical
num_classes = len(np.unique(train_label))
print("num_classes:",num_classes)
print(train_label[:5])
train_label = to_categorical(train_label)
test_label = to_categorical(test_label, num_classes)
val_label = to_categorical(val_label, num_classes)

# print shape of training set
print("num_classes:", num_classes)

# print number of training, validation, and test images
print(train_label.shape, "train samples")
print(test_label.shape, "test samples")
print(val_label.shape, "validation samples")

num_classes: 26
[20  4  3 15 14]
num_classes: 26
(223470, 26) train samples
(74491, 26) test samples
(74490, 26) validation samples


In [48]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(32, 32, 2)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='tanh'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='tanh'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(26, activation='softmax'))
model.summary()


/home/jeffrey/greek-letter-cnn/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 16)     │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 32)     │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 548,682 (2.09 MB)

 Trainable params: 548,682 (2.09 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [50]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best_1000.keras', verbose=1, 
                               save_best_only=True,monitor='val_accuracy', mode = "max")
hist = model.fit(train_data, train_label, batch_size=None, epochs=1000,
          validation_data=(val_data, val_label),callbacks=[checkpointer], 
          verbose=1, shuffle=True) 

Epoch 1/1000
6984/6984 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8811 - loss: 0.4170
Epoch 1: val_accuracy improved from -inf to 0.96123, saving model to model.weights.best_1000.keras
6984/6984 ━━━━━━━━━━━━━━━━━━━━ 38s 5ms/step - accuracy: 0.8811 - loss: 0.4170 - val_accuracy: 0.9612 - val_loss: 0.1426
Epoch 2/1000
6977/6984 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9339 - loss: 0.2712
Epoch 2: val_accuracy improved from 0.96123 to 0.96515, saving model to model.weights.best_1000.keras
6984/6984 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - accuracy: 0.9339 - loss: 0.2712 - val_accuracy: 0.9651 - val_loss: 0.1395
Epoch 3/1000
6983/6984 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9300 - loss: 0.3115
Epoch 3: val_accuracy did not improve from 0.96515
6984/6984 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step - accuracy: 0.9300 - loss: 0.3115 - val_accuracy: 0.9630 - val_loss: 0.1620
Epoch 4/1000
6973/6984 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9319 - loss: 0.3196
Epoch 4: val_accuracy did no

KeyboardInterrupt: 

In [ ]:
model.predict()